# Predictability of Mean Runoff

## Introduction

In the data preprocessing, we computed the entropy of the distribution of each individual streamflow time series in bits per sample.  We'll now use an ensemble decision tree method called gradient boosting using the XGBoost (eXtreme Gradient Boosted decision tree) {cite}`chen2016xgboost` library to see if runoff can be predicted from catchment attributes as was shown in {cite}`addor2018ranking`.  The model input features are added in successive model tests to compare the contribution of catchment attribute groups related to climate, terrain, land cover, and soil.  

In [1]:
import os
import pandas as pd
import numpy as np

from bokeh.plotting import figure, show
from bokeh.layouts import gridplot
from bokeh.models import ColumnDataSource
from bokeh.io import output_notebook
from bokeh.palettes import Sunset10, Vibrant7

import xgboost as xgb
from sklearn.metrics import (
    root_mean_squared_error,
    mean_absolute_error,
    roc_auc_score,
    accuracy_score,
)

import data_processing_functions as dpf

from scipy.stats import linregress
output_notebook()

Loading BokehJS ...

## Load Input Data

In [2]:
# load the catchment characteristics
fname = 'BCUB_HYSETS_properties_with_climate_with_entropy.csv'
df = pd.read_csv(os.path.join('data', fname))

Compute the mean runoff for each streamflow timeseries.

In [3]:
if 'mean_runoff' not in df.columns:
    for i, row in df.iterrows():
        mean_runoff = dpf.compute_mean_runoff(row)
        df.loc[i, 'mean_runoff'] = mean_runoff
    df.to_csv(os.path.join('data', fname))

Subdivide the attributes into related classes: terrain, land cover, soil, climate.

In [4]:
# list all the attributes in the input dataframe
print(df.columns.tolist())

['official_id', 'watershed_id', 'name', 'centroid_lat_deg_n', 'centroid_lon_deg_e', 'drainage_area_km2', 'drainage_area_gsim_km2', 'flag_gsim_boundaries', 'flag_artificial_boundaries', 'elevation_m', 'slope_deg', 'gravelius', 'perimeter', 'flag_shape_extraction', 'aspect_deg', 'flag_terrain_extraction', 'land_use_forest_frac_2010', 'land_use_grass_frac_2010', 'land_use_wetland_frac_2010', 'land_use_water_frac_2010', 'land_use_urban_frac_2010', 'land_use_shrubs_frac_2010', 'land_use_crops_frac_2010', 'land_use_snow_ice_frac_2010', 'flag_land_use_extraction', 'logk_ice_x100', 'porosity_x100', 'flag_subsoil_extraction', 'year_from', 'year_to', 'record_length', 'agency', 'status', 'updated_official_basin', 'in_bcub', 'prcp', 'srad', 'swe', 'tmax', 'tmin', 'vp', 'high_prcp_freq', 'high_prcp_duration', 'low_prcp_freq', 'low_prcp_duration', 'H_4_bits', 'H_6_bits', 'H_8_bits', 'H_9_bits', 'H_10_bits', 'H_11_bits', 'H_12_bits', 'mean_runoff']


## Define attribute groups

In [5]:
terrain = ['drainage_area_km2', 'elevation_m', 'slope_deg', 'gravelius', 'perimeter', 'aspect_deg']
land_cover = [
    'land_use_forest_frac_2010', 'land_use_grass_frac_2010', 'land_use_wetland_frac_2010', 'land_use_water_frac_2010', 
    'land_use_urban_frac_2010', 'land_use_shrubs_frac_2010', 'land_use_crops_frac_2010', 'land_use_snow_ice_frac_2010']
soil = ['logk_ice_x100', 'porosity_x100']
climate = ['prcp', 'srad', 'swe', 'tmax', 'tmin', 'vp', 'high_prcp_freq', 'high_prcp_duration', 'low_prcp_freq', 'low_prcp_duration']
all_attributes = terrain + land_cover + soil + climate
len(all_attributes)

26

In [6]:
results_folder = os.path.join('./data/', 'runoff_prediction_results')
if not os.path.exists(results_folder):
    os.makedirs(results_folder)

In [7]:
def predict_runoff_from_attributes(df, holdout_pct, group_order, results_folder):
    df.reset_index(drop=True, inplace=True)
    train_indices, test_indices = dpf.train_test_split(df, holdout_pct)
        
    # set the target column
    target_column = f'mean_runoff'
    test_attributes = []

    # add attribute groups successively
    for set_name in group_order:
        attribute_set = attribute_set_dict[set_name]
        print(f' Processing {set_name} attribute set')
        test_attributes += attribute_set
        input_data = df[test_attributes + [target_column]].copy()

        # run the XGBoost model with cross validation and test on holdout set
        trial_df, test_df = dpf.run_xgb_CV_trials(
            set_name, test_attributes, target_column, input_data, train_indices, 
            test_indices, n_optimization_rounds, nfolds, n_boost_rounds, results_folder
        )

        test_rmse = root_mean_squared_error(test_df['actual'], test_df['predicted'])
        test_mae = mean_absolute_error(test_df['actual'], test_df['predicted'])

        print(f'  {set_name}')
        print(f'   held-out test rmse: {test_rmse:.2f}, mae: {test_mae:.2f}')
        print('')
        # store the test set predictions and actuals
        all_test_results[set_name] = {
            'trials': trial_df, 'test_df': test_df,
            'test_mae': test_mae, 'test_rmse': test_rmse,
        } 
    return all_test_results

In [8]:
# define the amount of data to set aside for final testing
holdout_pct = 0.10
nfolds = 5
n_boost_rounds = 2000
n_optimization_rounds = 20

all_test_results = {}
attribute_set_dict = {
    'climate': climate, 
    '+land_cover': land_cover,
    '+terrain': terrain, 
    '+soil': soil,
}


In [9]:
def create_results_plots(all_test_results, attribute_sets):
    
    plots = []

    test_rmse, test_mae = [], []

    y1 = [all_test_results[e]['test_rmse'] for e in attribute_sets]
    y2 = [all_test_results[e]['test_mae'] for e in attribute_sets]

    source = ColumnDataSource({'x': attribute_sets, 'y1': y1, 'y2': y2})

    title = f'Runoff Predictability'

    if len(plots) == 0:
        fig = figure(title=title, x_range=attribute_sets)
    else:
        fig = figure(title=title, x_range=attribute_sets, y_range=plots[0].y_range)
    fig.line('x', 'y1', legend_label='rmse', color='green', source=source, line_width=3)
    fig.line('x', 'y2', legend_label='mae', color='dodgerblue', source=source, line_width=3)
    fig.legend.background_fill_alpha = 0.6
    fig.yaxis.axis_label = 'RMSE'

    result_df = pd.DataFrame({'set': attribute_sets, 'rmse': y1, 'mae': y2})
    best_rmse_idx = result_df['rmse'].idxmin()
    best_mae_idx = result_df['mae'].idxmin()
    best_rmse_set = result_df.loc[best_rmse_idx, 'set']
    best_mae_set = result_df.loc[best_mae_idx, 'set']
    best_result = all_test_results[best_rmse_set]['test_df']

    xx, yy = best_result['actual'], best_result['predicted']
    slope, intercept, r, p, se = linregress(xx, yy)

    sfig = figure(title=f'Test: best model {best_rmse_set} (N={len(best_result)})',
                 )
    sfig.scatter(xx, yy, size=3, alpha=0.8)
    x_obs = np.linspace(min(xx), max(xx), 1000)
    ybf = [slope * e + intercept for e in x_obs]
    sfig.line(x_obs, ybf, color='red', line_width=3, line_dash='dashed', legend_label=f'R²={r**2:.2f}')    
    sfig.xaxis.axis_label = r'$$\text{Observed Mean} \left[ m^3 / s \right]$$'
    sfig.yaxis.axis_label = r'$$\text{Predicted Mean} \left[ m^3 / s \right]$$'
    sfig.legend.location = 'top_left'
    plots.append(fig)
    plots.append(sfig)
    return plots

## Set Attribute Groupings

In [10]:
group_1 = ['climate', '+terrain', '+land_cover', '+soil']
group_2 = group_1[::-1]
group_3 = ['+land_cover', '+terrain', '+soil', 'climate']
group_4 = ['+soil', 'climate', '+land_cover', '+terrain']
attribute_group_orders = [group_1, group_2, group_3, group_4]

## Run XGBoost Models

In [11]:
n = 0
group_results = {}
for group in attribute_group_orders:
    print(f'Processing: {group} ordering.')
    n += 1
    test_results_fname = f'Mean_runoff_prediction_results_{n}.npy'
    test_results_fpath = os.path.join('data', test_results_fname)
    if os.path.exists(test_results_fpath):
        all_test_results = np.load(test_results_fpath, allow_pickle=True).item()
    else:
        all_test_results = predict_runoff_from_attributes(df, holdout_pct, group, results_folder)
        np.save(test_results_fpath, all_test_results)
    
    group_results[n] = {'order': group, 'results': all_test_results}
    

Processing: ['climate', '+terrain', '+land_cover', '+soil'] ordering.
 Processing climate attribute set
    48.70 ± 8.413 RMSE mean on the test set (N=20)
  climate
   held-out test rmse: 83.89, mae: 40.68

 Processing +terrain attribute set
    17.92 ± 4.137 RMSE mean on the test set (N=20)
  +terrain
   held-out test rmse: 25.92, mae: 15.48

 Processing +land_cover attribute set
    13.02 ± 2.741 RMSE mean on the test set (N=20)
  +land_cover
   held-out test rmse: 20.63, mae: 13.08

 Processing +soil attribute set
    13.18 ± 2.863 RMSE mean on the test set (N=20)
  +soil
   held-out test rmse: 37.06, mae: 14.00

Processing: ['+soil', '+land_cover', '+terrain', 'climate'] ordering.
 Processing +soil attribute set
    49.72 ± 4.015 RMSE mean on the test set (N=20)
  +soil
   held-out test rmse: 141.25, mae: 54.77

 Processing +land_cover attribute set
    31.52 ± 3.962 RMSE mean on the test set (N=20)
  +land_cover
   held-out test rmse: 94.03, mae: 32.63

 Processing +terrain attrib

## View Results

In [12]:
n = 1
grp_1_plots = create_results_plots(group_results[n]['results'], group_results[n]['results'])
layout = gridplot(grp_1_plots, ncols=2, width=350, height=300)
show(layout)

ValueError: failed to validate ColumnDataSource(id='p1001', ...).data: expected a dict of type ColumnData(String, Seq(Any)), got a dict with invalid values for keys: x

### Test the sensitivity to Order of attribute groups

In [ ]:
n = 2
grp_2_plots = create_results_plots(group_results[n]['results'], group_results[n]['results'])
layout = gridplot(grp_2_plots, ncols=2, width=350, height=300)
show(layout)

In [ ]:
n = 3
grp_3_plots = create_results_plots(group_results[n]['results'], group_results[n]['results'])
layout = gridplot(grp_3_plots, ncols=2, width=350, height=300)
show(layout)

In [ ]:
n = 4
grp_4_plots = create_results_plots(group_results[n]['results'], group_results[n]['results'])
layout = gridplot(grp_4_plots, ncols=2, width=350, height=300)
show(layout)

### Test randomly permuted target values

As a last iteration, randomize the order of the mean_runoff column to test what the algorithm is learning.

The predictive power decreases substantially across all groupings of input attributes.

In [ ]:
shuffled_df = df.copy()
runoff_values = df['mean_runoff'].values
# randomly shuffle the order of runoff values
np.random.shuffle(runoff_values)
shuffled_df['mean_runoff'] = runoff_values
shuffled_results = predict_runoff_from_attributes(shuffled_df, holdout_pct, group_order_1)

In [ ]:
shuffled_runoff_plots = create_results_plots(shuffled_results, group_order_1)
layout = gridplot(shuffled_runoff_plots, ncols=2, width=350, height=300)
show(layout)

## Citations

```{bibliography}
:filter: docname in docnames
```